In [ ]:
import market
import simulate
import withdrawal
import harvesting
import metrics
import lens
import plot
from decimal import Decimal as D
import pandas

%matplotlib inline
import seaborn
from matplotlib import pyplot as plt
import matplotlib
seaborn.set(style="whitegrid")
seaborn.set_context('poster')

In [ ]:
length = 10
rate = D('.015')
s = simulate.calc_lens(harvesting.N_80_RebalanceHarvesting,
                   lambda p, h: withdrawal.ConstantDollar(p, h, rate=rate),
                   length,
                   lambda n: n[-1].portfolio_post.value_r,
                   portfolio=(800_000, 200_000)
                  )
# it comes back as Decimal which seaborn doesn't like
s = s.astype(int)

In [ ]:
#g = s.astype(int).plot()
g = seaborn.relplot(data=s, aspect=3)
g.despine(left=True, bottom=True, offset=20)
#g.set_xticklabels(['4%', '4.5%', '5%'])
g.fig.suptitle(f'Final (real) portfolio value after {length} years of {rate} withdrawals.')
g.ax.yaxis.set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

In [ ]:
g = seaborn.relplot(data=s[s<1_000_000], aspect=3)
g.despine(left=True, bottom=True, offset=20)
#g.set_xticklabels(['4%', '4.5%', '5%'])
g.fig.suptitle(f'Final (real) portfolio value after {length} years of {rate} withdrawals.')
print(len(s), len(s[s<1_000_000]))
g.ax.yaxis.set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

In [ ]:
#g = s.astype(int).plot()
new_s = 35_000 / s
g = seaborn.relplot(data=new_s, aspect=3)
g.despine(left=True, bottom=True, offset=20)
#g.set_xticklabels(['4%', '4.5%', '5%'])
g.fig.suptitle(f'Final (real) portfolio value after {length} years of {rate} withdrawals.')

In [ ]:
#g = seaborn.relplot(data=new_s[new_s>0.035], aspect=3)
a = s[s<1_000_000]
a = 35_000 / a
a = a[a>0.035]
g = seaborn.relplot(data=a, aspect=3)
g.despine(left=True, bottom=True, offset=20)
g.fig.suptitle(f'Current withdrawal rate after {length} years of {rate} withdrawals.')

In [ ]:
s[s<1_000_000]

In [ ]:
single_s = simulate.calc_lens(harvesting.N_80_RebalanceHarvesting,
                   lambda p, h: withdrawal.ConstantDollar(p, h, rate=D('.035')),
                   55,
                   lens.calc_pwa0,
                   portfolio=(800_000, 200_000)
                  )

In [ ]:
single_s[single_s>0].min()

In [ ]:
f = [0] * 115
for value in s[s<1_000_000]:
    sy = simulate.calc_lens(harvesting.N_80_RebalanceHarvesting,
               lambda p, h: withdrawal.ConstantDollar(p, h, rate=D(35_000 / value)),
               45,
               lens.calc_shortfall_years,
               portfolio=(800_000, 200_000)
    )
    failures = sy[sy>0]
    f.append(len(failures) / len(sy))
fs = pandas.Series(f)

In [ ]:
1 - fs.mean()

In [ ]:
fs[fs>0].sort_values(ascending=False)

In [ ]:
len(fs[fs>0.4])/len(fs[fs>0])